In [14]:
%env DASHSCOPE_API_KEY=sk-057d8d26302f4d749a435c8be3458fc7

env: DASHSCOPE_API_KEY=sk-057d8d26302f4d749a435c8be3458fc7


In [15]:
from http import HTTPStatus
from dashscope import Generation

In [34]:
prompt='''你是重庆小雨点的一个信贷审核员，你需要根据用户的申请记录打电话与用户沟通确认用户的身份和贷款信息是否。
        接下来我们模拟电话沟通，我作为用户会逐一回答你的问题，请你模拟电话接通后开始逐一提问，我们一问一答，直到所有问题都经过确认。
        以下是示例：
        <|assistant|>：你好，我是重庆小雨点的审核员，请问是{用户名}吗？
        <|user|>：是的，有什么事情？
        <|assistant|>：我们收到了一笔贷款申请，想和你核实一下。通话有录音。请问你的身份证号后四位是什么？
        <|user|>：是{身份证号后四位}。
        <|assistant|>：好的，和申请记录一致。请问这次贷款申请是你本人发起的吗？
        <|user|>：是的。
        <|assistant|>：请问这次贷款申请的用途是什么？
        <|user|>：用来做美容
        <|assistant|>：请问你是在哪家美容机构进行的申请？
        <|user|>：{美容机构名}
        <|assistant|>：好的，我这边确认晚了。我将尽快将您的信息提交给相关部门进行进一步处理。请您耐心等待，一般情况下，我们会在尽快的时间内通知您关于贷款批准或者需要提供额外信息的情况。如果您有其他紧急的问题或者需要了解更多信息，请随时告诉我，我将尽力帮助您。再见！
        <|user|>：好的，再见'''

In [35]:
messages = [
        {'role': 'system', 'content': '''
            以下是用户的申请记录：
            申请人：王美丽
            申请时间：2023年12月13日
            申请原因：光子嫩肤
            美容机构：华韩整形重庆分院
            身份证号：500101198212156725
            '''}]

In [36]:
def call_with_stream(prompt, messages):
    responses = Generation.call(
        Generation.Models.qwen_turbo,
        messages=messages,
        prompt=prompt,
        result_format='message',  # set the result to be "message" format.
        stream=True,
        incremental_output=True  # get streaming output incrementally
    )
    full_content = ''  # with incrementally we need to merge output.
    for response in responses:
        if response.status_code == HTTPStatus.OK:
            full_content += response.output.choices[0]['message']['content']
            print(response)
        else:
            print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message
            ))
    print('Full response:\n' + full_content)
    return full_content

In [37]:
full_content = call_with_stream(prompt,messages)

{"status_code": 200, "request_id": "1a7d5c64-4a3f-9b5c-b3a5-938df3770c05", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "null", "message": {"role": "assistant", "content": "<|assistant"}}]}, "usage": {"input_tokens": 402, "output_tokens": 3, "total_tokens": 405}}
{"status_code": 200, "request_id": "1a7d5c64-4a3f-9b5c-b3a5-938df3770c05", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "null", "message": {"role": "assistant", "content": "|>：您好，我是重庆小雨点的"}}]}, "usage": {"input_tokens": 402, "output_tokens": 14, "total_tokens": 416}}
{"status_code": 200, "request_id": "1a7d5c64-4a3f-9b5c-b3a5-938df3770c05", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "null", "message": {"role": "assistant", "content": "信贷审核员，请问您是王美丽女士"}}]}, "usage": {"input_tokens": 402, "output_tokens": 24, "total_tokens": 426}}
{"status_code": 200

In [38]:
import re

def generate_speech_for_sentence(history, chatbot_role, sentence, return_as_byte=False):
    language = "autodetect"

    wav_bytestream = b""
    
    if len(sentence)==0:
        print("EMPTY SENTENCE")
        return 
    
    # Sometimes prompt </s> coming on output remove it
    # Some post process for speech only
    sentence = sentence.replace("</s>", "")
    # remove code from speech
    sentence = re.sub("```.*```", "", sentence, flags=re.DOTALL)
    sentence = re.sub("`.*`", "", sentence, flags=re.DOTALL)
    
    sentence = re.sub("\(.*\)", "", sentence, flags=re.DOTALL)
    
    sentence = sentence.replace("```", "")
    sentence = sentence.replace("...", " ")
    sentence = sentence.replace("(", " ")
    sentence = sentence.replace(")", " ")
    sentence = sentence.replace("<|assistant|>","")

    if len(sentence)==0:
        print("EMPTY SENTENCE after processing")
        return 
        
    # A fast fix for last chacter, may produce weird sounds if it is with text
    #if (sentence[-1] in ["!", "?", ".", ","]) or (sentence[-2] in ["!", "?", ".", ","]):
    #    # just add a space
    #    sentence = sentence[:-1] + " " + sentence[-1]
        
    # regex does the job well
    sentence= re.sub("([^\x00-\x7F]|\w)(\.|\。|\?|\!)",r"\1 \2\2",sentence)
    
    print("Sentence for speech:", sentence)

    
    try:
        SENTENCE_SPLIT_LENGTH=350
        if len(sentence)<SENTENCE_SPLIT_LENGTH:
            # no problem continue on
            sentence_list = [sentence]
        else:
            # Until now nltk likely split sentences properly but we need additional 
            # check for longer sentence and split at last possible position
            # Do whatever necessary, first break at hypens then spaces and then even split very long words
            sentence_list=textwrap.wrap(sentence,SENTENCE_SPLIT_LENGTH)
            print("SPLITTED LONG SENTENCE:",sentence_list)
        
        for sentence in sentence_list:
            
            if any(c.isalnum() for c in sentence):
                if language=="autodetect":
                    #on first call autodetect, nexts sentence calls will use same language
                    language = detect_language(sentence) 
            
                #exists at least 1 alphanumeric (utf-8) 
                audio_stream = get_voice_streaming(
                        sentence, language, latent_map[chatbot_role]
                    )
            else:
                # likely got a ' or " or some other text without alphanumeric in it
                audio_stream = None 
                
            # XTTS is actually using streaming response but we are playing audio by sentence
            # If you want direct XTTS voice streaming (send each chunk to voice ) you may set DIRECT_STREAM=1 environment variable
            if audio_stream is not None:
                frame_length = 0
                for chunk in audio_stream:
                    try:
                        wav_bytestream += chunk
                        frame_length += len(chunk)
                    except:
                        # hack to continue on playing. sometimes last chunk is empty , will be fixed on next TTS
                        continue

            # Filter output for better voice
            filter_output=False 
            if filter_output:
                data_s16 = np.frombuffer(wav_bytestream, dtype=np.int16, count=len(wav_bytestream)//2, offset=0)
                float_data = data_s16 * 0.5**15
                reduced_noise = nr.reduce_noise(y=float_data, sr=24000,prop_decrease =0.8,n_fft=1024)
                wav_bytestream = (reduced_noise * 32767).astype(np.int16)
                wav_bytestream = wav_bytestream.tobytes()
                    
            if audio_stream is not None:
                if not return_as_byte:
                    audio_unique_filename = "/tmp/"+ str(uuid.uuid4())+".wav"
                    with wave.open(audio_unique_filename, "w") as f:
                        f.setnchannels(1)
                        # 2 bytes per sample.
                        f.setsampwidth(2)
                        f.setframerate(24000)
                        f.writeframes(wav_bytestream)
                           
                    return (history , gr.Audio.update(value=audio_unique_filename, autoplay=True))
                else:
                    return (history , gr.Audio.update(value=wav_bytestream, autoplay=True))
    except RuntimeError as e:
        if "device-side assert" in str(e):
            # cannot do anything on cuda device side error, need tor estart
            print(
                f"Exit due to: Unrecoverable exception caused by prompt:{sentence}",
                flush=True,
            )
            gr.Warning("Unhandled Exception encounter, please retry in a minute")
            print("Cuda device-assert Runtime encountered need restart")

            # HF Space specific.. This error is unrecoverable need to restart space
            api.restart_space(repo_id=repo_id)
        else:
            print("RuntimeError: non device-side assert error:", str(e))
            raise e

    print("All speech ended")
    return 
